In [1]:
import json

In [2]:
with open('pmc_vqa_nota1K.json') as f:
    PMC=json.loads(f.read())

with open('swap_img_shuffle.json') as f:
    SWAP=[v for _, v in json.loads(f.read()).items()]

with open('fake_qa_shuffle.json') as f:
    FAKE=[v for _, v in json.loads(f.read()).items()]

In [4]:
def pmc_vqa(d, prompt='SIMPLE'):
    def extract_choices(i):
        return ' '.join([i['choice_A'].replace('A: ', 'A:').replace('A:', 'A. ').strip(),
                 i['choice_B'].replace('B: ', 'B:').replace('B:', 'B. ').strip(),
                 i['choice_C'].replace('C: ', 'C:').replace('C:', 'C. ').strip(),
                 i['choice_D'].replace('D: ', 'D:').replace('D:', 'D. ').strip(),])
    pmc_d=[{'image': i['img'], 
      'text': PROMPT_DICT[prompt].format(question=i['question'].strip(), option=extract_choices(i)),
      'answer': i['answer'].strip(),
      'question_id': idx
     } for idx, i in enumerate(d)]

    with open(f'./{prompt}/pmc_vqa_nota.jsonl', 'w') as fi:
        fi.writelines([json.dumps(i)+"\n" for i in pmc_d])

In [17]:
def swap(d, prompt='SIMPLE'):
    def extract_choices(i):
        return ' '.join([f'{k.strip()}. {o.strip()}' for k, o in i['option'].items()])
    swap_d=[{'image': i['img'], 
      'text': PROMPT_DICT[prompt].format(question=i['question'].strip(), option=extract_choices(i)),
      'answer': i['answer_id'].strip(),
      'question_id': idx
     } for idx, i in enumerate(d)]

    with open(f'./{prompt}/swap_img_shuffle.jsonl', 'w') as fi:
        fi.writelines([json.dumps(i)+"\n" for i in swap_d])

In [18]:
def fake(d, prompt='SIMPLE'):
    def extract_choices(i):
        return ' '.join([f'{k.strip()}. {o.strip()}' for k, o in i['option'].items()])
    fake_d=[{'image': i['img'], 
      'text': PROMPT_DICT[prompt].format(question=i['question'].strip(), option=extract_choices(i)),
      'answer': i['answer_id'].strip(),
      'question_id': idx
     } for idx, i in enumerate(d)]

    with open(f'./{prompt}/fake_qa_shuffle.jsonl', 'w') as fi:
        fi.writelines([json.dumps(i)+"\n" for i in fake_d])

In [51]:
PROMPT_DICT = {
'SIMPLE':(
        "{question} {option}."
    ),
'SEPARATOR':(
        "### Question:\n{question}\n\n### Choices:\n{option}\n\n### Answer:"
    ),
'ONLY':(
        "{question} {option}. only give me one token of the answer, no other words."
    ),
'LETTER':(
        "Answer with the option's letter from the given choices directly.\n\n{question} {option}."
    ),
'LETSEP':(
        "Answer with the option's letter from the given choices directly.\n\n"
        "### Question:\n{question}\n\n### Choices:\n{option}\n\n### Answer:"
    ),
'RP0LETTER':(
        "You are a medical doctor and expert in medical imaging. "
        "Answer with the option's letter from the given choices directly.\n\n{question} {option}."
    ),
'RP1LETTER':(
        "Act as a medical domain expert answering multiple-choice questions. "
        "Answer with the option's letter from the given choices directly.\n\n{question} {option}."
    ),
'ACCU0LETTER':(
        "Always answer accurately and precisely. "
        "Answer with the option's letter from the given choices directly.\n\n{question} {option}."
    ),
'ACCU1LETTER':(
        "Your answer should be precise and free of incomplete or incorrect biomedical or clinical information. "
        "Answer with the option's letter from the given choices directly.\n\n{question} {option}."
    ),
'DONT0LETTER':(
        "If you don't know the answer to a question, please don't share false information. "    
        "Answer with the option's letter from the given choices directly.\n\n{question} {option}."
    ),
'DONT1LETTER':(
        "If you do not know the answer, do not try to make up an answer. "    
        "Answer with the option's letter from the given choices directly.\n\n{question} {option}."
    ),
'ACCU1RP0LETTER':(
        "You are a medical doctor and expert in medical imaging. "
        "Your answer should be precise and free of incomplete or incorrect biomedical or clinical information. "
        "Answer with the option's letter from the given choices directly.\n\n{question} {option}."
    ),
'ACCU1DONT0LETTER':(
        "Your answer should be precise and free of incomplete or incorrect biomedical or clinical information. "
        "If you don't know the answer to a question, please don't share false information. "    
        "Answer with the option's letter from the given choices directly.\n\n{question} {option}."
    ),
'RP0DONT0LETTER':(
        "You are a medical doctor and expert in medical imaging. "
        "If you don't know the answer to a question, please don't share false information. "    
        "Answer with the option's letter from the given choices directly.\n\n{question} {option}."
    ),
'ALL':(
        "You are a medical doctor and expert in medical imaging. "
        "Your answer should be precise and free of incomplete or incorrect biomedical or clinical information. "
        "If you don't know the answer to a question, please don't share false information. "    
        "Answer with the option's letter from the given choices directly.\n\n{question} {option}."
    ),
'ALLSEP':(
        "You are a medical doctor and expert in medical imaging. "
        "Your answer should be precise and free of incomplete or incorrect biomedical or clinical information. "
        "If you don't know the answer to a question, please don't share false information. "    
        "Answer with the option's letter from the given choices directly.\n\n"
        "### Question:\n{question}\n\n### Choices:\n{option}\n\n### Answer:"
    ),
'RP0DONT0LETSEP':(
        "You are a medical doctor and expert in medical imaging. "
        "If you don't know the answer to a question, please don't share false information. "    
        "Answer with the option's letter from the given choices directly.\n\n"
        "### Question:\n{question}\n\n### Choices:\n{option}\n\n### Answer:"
    ),
'DONT0LETSEP':(
        "If you don't know the answer to a question, please don't share false information. "    
        "Answer with the option's letter from the given choices directly.\n\n"
        "### Question:\n{question}\n\n### Choices:\n{option}\n\n### Answer:"
    ),
}

In [52]:
import os

In [53]:
for i in PROMPT_DICT:
    os.makedirs(f'./{i}', exist_ok=True)
    os.makedirs(f'/working/llms/MULTIMODAL/models/llava/hallucination_test/{i}', exist_ok=True)
    pmc_vqa(PMC, prompt=i)
    swap(SWAP, prompt=i)
    fake(FAKE, prompt=i)

In [26]:
def extract_choices(i):
    return ' '.join([f'{k.strip()}. {o.strip()}' for k, o in i['option'].items()])

In [27]:
fake_d=[{'image': i['img'], 
  'text': PROMPT_DICT['prompt'].format(question=i['question'].strip(), option=extract_choices(i)),
  'answer': i['answer_id'].strip(),
  'question_id': idx
 } for idx, i in enumerate(d)]

In [28]:
with open('fake_qa_shuffle.jsonl', 'w') as fi:
    fi.writelines([json.dumps(i)+"\n" for i in swap_d])